# _Exploratory Data Analysis of the Population and Crop production data._

## Dataset Summary.

There are 2 datasets used in this project; Population dataset and Crops dataset. The population dataset used in this project was obtained from the **WORLD BANK GROUP** and it consists of the population of all countries and economies ranging from the years 1960 to 2019. This dataset was obtained from [this link](https://data.worldbank.org/indicator/SP.POP.TOTL), specifically [this file](http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv). The world bank group makes this data publicly available according to the open data standards and licenses datasets under the Creative Commons Attribution 4.0 International license (CC-BY 4.0). 


Furthermore, the crops dataset used in this project was acquired from the **Food and Agriculture Organization of the United Nations (FAO)**

# _References_

Data.worldbank.org. 2020. Population, Total | Data. [online] Available at: <https://data.worldbank.org/indicator/SP.POP.TOTL> [Accessed 21 November 2020].